## Extraer datos mediante la API de inmobiliaria

pagina donde se hizo el webscraping: https://www.argenprop.com/

## Paises admitidos:
#### Argentina, Chile, Uruguay, Brasil

ojo: brasil solo tiene 1 propiedad

## 1. Creamos la conexión con la API para extraer los datos

In [1]:
%%time
import requests
import pandas as pd

url = 'http://127.0.0.1:5000/argenprop'

# Parámetros de la solicitud
params = {'pais': 'argentina', 'limite': 300}
response = requests.get(url, params=params)

data = response.json()
df = pd.DataFrame(data)

CPU times: total: 359 ms
Wall time: 3min 50s


In [12]:
from geopy.geocoders import Nominatim
import numpy as np

In [6]:
df.head()

,Bathrooms,Bedrooms,CountryName,Description,Environments,Latitude,Longitude,Price,Seniority,StateName,Title,img,address
0,2,3,Argentina,"Casa en Barrio Privado Camino Real, cuenta con...",4,"-34,48684","-58,58234",USD 2.300,27,Buenos Aires,Av. Camino Real Morón San Fernando 1500,[https://static1.sosiva451.com/08910661/ea4bac...,Error: Must be a coordinate pair or Point
1,2,3,Argentina,Alquilo amplio Duplex en Florencio varela comb...,4,"-34,79154","-58,27525",$ 490.000,10,Buenos Aires,Combate De San Lorenzo 1700,[https://static1.sosiva451.com/28339561/734f48...,Error: Must be a coordinate pair or Point
2,2,4,Argentina,DUEÑO ALQUILA\r\n\n •SIN EXPENSAS\r\n •SIN GAS...,5,None,None,USD 1.100,15,Buenos Aires,casa en lujan con pileta alquiler anual,[https://static1.sosiva451.com/09149461/6abd53...,"Dr. Gelfant, 1333, V6H 4C1"
3,2,3,Argentina,ALQUILER DE CASA EN BARRIO SANTA ELISA CON PIL...,4,"-34,394108","-58,89099",USD 1.500,0,Buenos Aires,Barrio Santa Elisa,[https://static1.sosiva451.com/43958561/2e68e4...,Error: Must be a coordinate pair or Point
4,4,4,Argentina,EL VALOR ES CON EXPENSAS INCLUIDAS.\n Excelent...,5,"-34,446335","-58,855007",USD 1.950,35,Buenos Aires,U$S1950 con expensas incluidas,[https://static1.sosiva451.com/04247161/263253...,Error: Must be a coordinate pair or Point


In [8]:
df['Latitude'] = df['Latitude'].str.replace(',', '.').astype(float)
df['Longitude'] = df['Longitude'].str.replace(',', '.').astype(float)


In [9]:
df = df.dropna(subset=['Latitude', 'Longitude'])


In [ ]:
# Inicializa el geocodificador
geolocator = Nominatim(user_agent="geoapi_exercises")

# Función para obtener la dirección
def obtener_direccion(lat, lon):
    try:
        location = geolocator.reverse((lat, lon))
        return location.address if location else "Dirección no encontrada"
    except Exception as e:
        return f"Error: {e}"

# Aplica la función a cada fila del DataFrame
df['Address'] = df.apply(lambda x: obtener_direccion(x['Latitude'], x['Longitude']), axis=1)

E:\Usuario\Archivos temporales de Usuario\ipykernel_17160\2421805109.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['address'] = df.apply(lambda x: obtener_direccion(x['Latitude'], x['Longitude']), axis=1)


In [ ]:
# me quedo con los primeros 3 elementos de la dirección
df['Address'] = df['Address'].apply(lambda x: ', '.join(x.split(', ')[:3]))

E:\Usuario\Archivos temporales de Usuario\ipykernel_17160\1926705852.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['address'] = df['address'].apply(lambda x: ', '.join(x.split(', ')[:3]))


In [ ]:
# Lista de columnas
columnas = [
    "Water", "Gas", "Surveillance", "Electricity", "Internet", 
    "Pool", "Garage", "Pets", "Grill", "Elevator", "Terrace",
    "IsHistoric", "IsWorking", "HasWarranty", "RangeSalary"
]

# Agregar columnas con valores específicos
for column in columnas:
    if column in ["Water", "Gas", "Electricity"]:
        df[column] = True  # Estas columnas son siempre True
    elif column in ["Surveillance", "Pets", "Pool"]:
        df[column] = np.random.choice([True, False], size=len(df))
    elif column == "RangeSalary":
        # Genero los salarios con distribución sesgada a la izquierda
        salary_range = np.random.triangular(left=400000, mode=1200000, right=3000000, size=len(df))
        salary_range = salary_range.astype(int)  # Asegurarnos de que sean números enteros
        df[column] = salary_range
    else:
        df[column] = np.random.choice([True, False], size=len(df), p=[0.8, 0.2]) # sesgo para que haya mas true que false

In [20]:
# elimino las columnas latitud y longitud
df = df.drop(columns=['Latitude', 'Longitude'])

In [22]:
orden_col = [
    "IsWorking", "HasWarranty", "RangeSalary", "CountryName", "StateName", "Title", "Address", "Price", "Environments", 
    "Bathrooms", "Bedrooms", "Seniority", "Water", "Gas", 
    "Surveillance", "Electricity", "Internet", "Pool", 
    "Garage", "Pets", "Grill", "Elevator", "Terrace", 
    "IsHistoric", "Description", "img"
]

# Reordenar las columnas del DataFrame
df = df[orden_col]

In [ ]:
df.head()